<a href="https://colab.research.google.com/github/RealLord/NLP_Partkom/blob/master/Partkom_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 2.x
import datetime

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from sklearn.metrics import confusion_matrix

import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Flatten, Activation, Dropout, GlobalMaxPool1D, Conv1D
from keras.layers import Embedding
from keras.preprocessing import text, sequence
from keras.preprocessing.text import Tokenizer
#from keras.utils.data_utils import pad_sequences
from keras.preprocessing.sequence import pad_sequences
from keras import utils
from google.colab import drive



Mounted at /content/drive/


In [2]:
def pir(s):
    return pd.DataFrame({'a':s.value_counts(), 'per':s.value_counts(normalize=True).mul(100).round(1).astype(str) + '%'})

In [25]:
def load_data():
    df_draft = pd.read_csv('https://github.com/RealLord/NLP_Partkom/raw/master/goods_01.zip', sep=';', header=0, encoding='utf-8', dtype={"id": int, "group_code": int, "desc": "string"}, compression="zip")
    return df_draft


In [26]:
df_draft = load_data()
df_draft.head(10)

,id,group_code,desc
0,0,719,834 A0009808164 амортизатор капота
1,1,299,722 TA1180 тяга рулевая
2,2,298,722 TA1186 наконечник
3,3,349,722 LM80113 цилиндр тормозной
4,4,688,740 09299 насос омывателя
5,5,609,696 1006209642 бендиксы стартера
6,6,663,834 0011592501 свеча накаливания
7,7,663,834 0011593501 свеча накаливания
8,8,663,852 DPOWER1 свеча накаливания alfaromeo 125bmw...
9,9,711,722 LL40811 рабочий цилиндр сцепления 2381 mm


In [5]:
sentences = df_draft['desc']
y = df_draft['group_code']

sentences_train, sentences_test, train_y, test_y = train_test_split(sentences, y, test_size=0.30, stratify=y)

tokenize = Tokenizer(num_words=5000)
texts = pd.concat([sentences_train], axis=0).astype("str")  # оставляем только тренировочную выборку
tokenize.fit_on_texts(texts)

X_train = tokenize.texts_to_sequences(pd.concat([sentences_train], axis=0).astype("str"))
X_test = tokenize.texts_to_sequences(pd.concat([sentences_test], axis=0).astype("str"))
vocab_size = len(tokenize.word_index) + 1

maxlen = 100
X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

# вариант с энкодингом
encoder = LabelEncoder()
encoder.fit(train_y)

y_train = encoder.transform(train_y)
y_test = encoder.transform(test_y)
num_classes = np.max(y_train) + 1
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test  = tf.keras.utils.to_categorical(y_test,  num_classes)


In [17]:
try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.TPUStrategy(tpu)

Running on TPU  ['10.15.28.114:8470']
INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.15.28.114:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.15.28.114:8470


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [15]:
%tensorflow_version 2.x
device_name = tf.test.gpu_device_name()
print(device_name)


def cpu():
  with tf.device('/cpu:0'):
    random_image_cpu = tf.random.normal((100, 100, 100, 3))
    net_cpu = tf.keras.layers.Conv2D(32, 7)(random_image_cpu)
    return tf.math.reduce_sum(net_cpu)

def gpu():
  with tf.device('/device:GPU:0'):
    random_image_gpu = tf.random.normal((100, 100, 100, 3))
    net_gpu = tf.keras.layers.Conv2D(32, 7)(random_image_gpu)
    return tf.math.reduce_sum(net_gpu)
  
# We run each op once to warm up; see: https://stackoverflow.com/a/45067900
cpu()
gpu()


<tf.Tensor: shape=(), dtype=float32, numpy=-48.26703>

In [22]:
###################################### CNN #######################################
embedding_dim = 30
with tpu_strategy.scope(): # creating the model in the TPUStrategy scope means we will train the model on the TPU
  model5 = Sequential()
  model5.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=maxlen))
  model5.add(Conv1D(128, 16, activation='relu'))  #  128 5 default
  model5.add(GlobalMaxPool1D())
  model5.add(Dense(1000, activation='relu'))  # 10 default
  model5.add(Dense(431, activation='softmax'))

  model5.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
  model5.summary()

#with tf.device("/gpu:0"):
  history_5 = model5.fit(X_train, y_train,
                batch_size=64,
                epochs=1,
                validation_data=(X_test, y_test))

with tf.device("/cpu:0"):
  loss, accuracy = model5.evaluate(X_train, y_train, verbose=False)
  print("Training Accuracy: {:.4f}".format(accuracy))
  loss, accuracy = model5.evaluate(X_test, y_test, verbose=False)
  print("Testing Accuracy:  {:.4f}".format(accuracy))

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_8 (Embedding)     (None, 100, 30)           38792070  
                                                                 
 conv1d_8 (Conv1D)           (None, 85, 128)           61568     
                                                                 
 global_max_pooling1d_8 (Glo  (None, 128)              0         
 balMaxPooling1D)                                                
                                                                 
 dense_16 (Dense)            (None, 1000)              129000    
                                                                 
 dense_17 (Dense)            (None, 431)               431431    
                                                                 
Total params: 39,414,069
Trainable params: 39,414,069
Non-trainable params: 0
__________________________________________

In [24]:
  #with tf.device("/cpu:0"):
  #  s = f'{datetime.datetime.now():%Y_%m_%d_%H_%M}'
  #  model5.save('/content/gdrive/My Drive/Partkom/model5.save.'+s)  # Saving model!    

INFO:tensorflow:Assets written to: /content/gdrive/My Drive/Partkom/model5.save.2022_07_01_12_32/assets


INFO:tensorflow:Assets written to: /content/gdrive/My Drive/Partkom/model5.save.2022_07_01_12_32/assets


UnimplementedError: ignored